In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizVert_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,2275.56,-0.01,0.03,-0.00,0.00,-0.00,0.00
3,-0.01,731.82,-0.01,0.01,0.00,0.00,-0.00
4,0.03,-0.01,204.85,0.00,-0.00,0.00,-0.00
5,-0.00,0.01,0.00,47.33,-0.00,-0.00,0.00
6,0.00,0.00,-0.00,-0.00,9.36,0.00,0.00
7,-0.00,0.00,0.00,-0.00,0.00,1.71,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.26


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,-0.00001,0.00005,-0.00000,0.00001,-0.00006,0.00001
3,-0.00001,1.00000,-0.00001,0.00003,0.00000,0.00008,-0.00005
4,0.00005,-0.00001,1.00000,0.00003,-0.00001,0.00003,-0.00001
5,-0.00000,0.00003,0.00003,1.00000,-0.00001,-0.00001,0.00003
6,0.00001,0.00000,-0.00001,-0.00001,1.00000,0.00002,0.00006
7,-0.00006,0.00008,0.00003,-0.00001,0.00002,1.00000,-0.00001
8,0.00001,-0.00005,-0.00001,0.00003,0.00006,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.05107406040057062

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.42268755e+05 7.44199042e+04 5.59535065e+03 2.80263054e+02
 9.76868330e+00 2.48668296e-01 4.19620366e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998121,-0.059369,-0.014741,-0.003380,-0.000667,-0.000124,-0.000019
1,0.058310,0.996322,-0.061395,-0.012906,-0.002541,-0.000462,-0.000072
2,0.018033,0.059498,0.996095,-0.061612,-0.011404,-0.002082,-0.000319
3,0.005163,0.016093,0.059713,0.995614,-0.068955,-0.012001,-0.001827
4,0.001349,0.004197,0.015014,0.066638,0.993066,-0.094582,-0.013723
5,0.000367,0.001132,0.004055,0.017574,0.090355,0.983944,-0.152848
6,0.000114,0.000354,0.001263,0.005464,0.027636,0.150861,0.988153


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00187869, 0.00367809, 0.0039055 , 0.00438635, 0.00693383,
       0.01605554, 0.01184736])